In [ ]:
# 文件c1、c4、c6为训练数据，文件c2、c3、c5为测试数据:
#
# 第1列:X维力(N)
# 第2列:Y维力(N)
# 第3列:Z维力(N)
# 第4列:X维振动(g)
# 第5列:Y维振动(g)
# 第6列:Z维振动(g)
# 第7列:AE-RMS (V)
#
# 刀具主轴转速为10400 RPM;进给速度1555 mm/min;切割Y深度(径向)为0.125 mm;
# Z轴向切割深度为0.2 mm。数据以50khz /通道采集。

import numpy as np
import pandas as pd
import glob
import os
import time
import matplotlib.pyplot as plt
from scipy.fftpack import fft

# 获取数据文件列表
class GetDataFiles():
    def __init__(self, path):
        self.path = path

    def get_datafiles(self):
        files = glob.glob(os.path.join(self.path, "c_*_*.csv"))
        return files


# 时域处理
class TimeDomainProcessing():
    def __init__(self, data):
        self.data = data

    # 求均值
    def get_Mean(self, i, k):
        k.append(np.mean(self.data.iloc[:, i]))
        return k

    # 求均方值
    def get_MeanSquareValue(self, i, k):
        a = self.get_Mean(i, k)[-1]
        b = self.get_Variance(i, k)[-1]
        k.append(pow(a, 2) + b)
        return k

    # 求均方根值
    def get_RMS(self, i, k):
        k.append(pow(self.get_MeanSquareValue(i, k)[-1], 0.5))
        return k

    # 求方差
    def get_Variance(self, i, k):
        k.append(np.var(self.data.iloc[:, i]))
        return k

    # 求标准差
    def get_StandardDeviation(self, i, k):
        k.append(pow(self.get_Variance(i, k)[-1], 0.5))
        return k

    # 求峰值
    def get_Peak(self, i, m, n):
        m.append(np.max(self.data.iloc[:, i]))
        n.append(np.min(self.data.iloc[:, i]))
        return m, n

    # 求偏斜度
    def get_Degree_of_Skewness(self, i, k):
        sum = 0
        n = np.size(self.data.iloc[:, i])
        mean = self.get_Mean(i, k)[-1]
        for j in range(n):
            sum = sum + pow((self.data.iloc[j, i] - mean), 3)
        k.append(sum/n)
        return k

    # 求峭度
    def get_Kurtosis(self, i, k):
        sum = 0
        n = np.size(self.data.iloc[:, i])
        mean = self.get_Mean(i, k)[-1]
        for j in range(n):
            sum = sum + pow((self.data.iloc[j, i] - mean), 4)
        k.append(sum/n)
        return k


    # 偏斜度指标
    def get_DS_index(self, i, k):
        a = self.get_Degree_of_Skewness(i, k)[-1]
        b = self.get_StandardDeviation(i, k)[-1]
        k.append(a/pow(b, 3))
        return k


    # 峭度指标
    def get_Kurtosis_index(self, i, k):
        a = self.get_Kurtosis(i, k)[-1]
        b = self.get_StandardDeviation(i, k)[-1]
        k.append(a/pow(b, 4))
        return k

    # 波形指标
    def get_Waveform_index(self, i, k):
        a = self.get_RMS(i, k)[-1]
        b = self.get_Mean(i, k)[-1]
        k.append(a/b)
        return k

    # 峰值指标
    def get_Peak_index(self, i, k):
        a = np.max(self.data.iloc[:, i])
        b = self.get_RMS(i, k)[-1]
        k.append(a/b)
        return k

    # 脉冲指标
    def get_Pulse_index(self, i, k):
        a = np.max(self.data.iloc[:, i])
        b = self.get_Mean(i, k)[-1]
        k.append(a/b)
        return k

    # 裕度指标
    def get_Margin_index(self, i, k):
        sum = 0
        a = np.max(self.data.iloc[:, i])
        n = np.size(self.data.iloc[:, i])
        for j in range(n):
            sum = sum + pow((abs(self.data.iloc[j, i])), 0.5)
        b = pow((sum/n), 2)
        k.append(a/b)
        return k

    # 求互相关函数
    def get_CrossCorrelation(self, data1, data2):
        cor = np.correlate(data1, data2, 'same')
        return cor

    # 特征值显示图像
    def get_CVimage(self, y, str):
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        plt.subplot(2, 3, 1)
        plt.plot(y[0], color="blue", linewidth=2.5, linestyle="-", label="F_x")
        plt.title('F_x的' + str)
        plt.subplot(2, 3, 2)
        plt.plot(y[1], color="blue", linewidth=2.5, linestyle="-", label="F_y")
        plt.title('F_y的' + str)
        plt.subplot(2, 3, 3)
        plt.plot(y[2], color="blue", linewidth=2.5, linestyle="-", label="F_z")
        plt.title('F_z的' + str)
        plt.subplot(2, 3, 4)
        plt.plot(y[3], color="blue", linewidth=2.5, linestyle="-", label="A_x")
        plt.title('A_x的' + str)
        plt.subplot(2, 3, 5)
        plt.plot(y[4], color="blue", linewidth=2.5, linestyle="-", label="A_y")
        plt.title('A_y的' + str)
        plt.subplot(2, 3, 6)
        plt.plot(y[5], color="blue", linewidth=2.5, linestyle="-", label="A_z")
        plt.title('A_z的' + str)
        plt.savefig('D:\\image\\' + str +'.png')
        plt.show()

    # 互相关函数显示图像
    def get_CCimage(self, y, str):
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        plt.subplot(2, 3, 1)
        plt.plot(y[0], color="blue", linewidth=2.5, linestyle="-", label="F_x")
        plt.title('F_x的' + str + '与磨损的互相关函数')
        plt.subplot(2, 3, 2)
        plt.plot(y[1], color="blue", linewidth=2.5, linestyle="-", label="F_y")
        plt.title('F_y的' + str + '与磨损的互相关函数')
        plt.subplot(2, 3, 3)
        plt.plot(y[2], color="blue", linewidth=2.5, linestyle="-", label="F_z")
        plt.title('F_z的' + str + '与磨损的互相关函数')
        plt.subplot(2, 3, 4)
        plt.plot(y[3], color="blue", linewidth=2.5, linestyle="-", label="A_x")
        plt.title('A_x的' + str + '与磨损的互相关函数')
        plt.subplot(2, 3, 5)
        plt.plot(y[4], color="blue", linewidth=2.5, linestyle="-", label="A_y")
        plt.title('A_y的' + str + '与磨损的互相关函数')
        plt.subplot(2, 3, 6)
        plt.plot(y[5], color="blue", linewidth=2.5, linestyle="-", label="A_z")
        plt.title('A_z的' + str + '与磨损的互相关函数')
        plt.savefig('D:\\image\\' + str + '.png')
        plt.show()


# 频域处理
class FrequencyDomainProcessing():
    def __init__(self, data, Fs):
        self.data = data
        self.Fs = Fs

    # 求快速傅里叶变换（FFT）
    def get_FFT(self, i):
        L = len(self.data.iloc[:, i])
        k = self.Fs / L
        # N = np.power(2, np.ceil(np.log2(L)))
        FFT_y = abs(fft(self.data.iloc[:, i].values))/L * 2
        Fre = np.arange(L // 2) * k
        FFT_y = FFT_y[range(int(L / 2))]
        return Fre, FFT_y

    # 求自功率谱密度函数
    def get_SelfPowerSpectrumDensityFunction(self, i):
        L = len(self.data.iloc[:, i])
        cor_x = np.correlate(data.iloc[:, i], data.iloc[:, i], 'same')
        cor_X = fft(cor_x, L)
        ps_cor = np.abs(cor_X)
        ps_cor = ps_cor/np.max(ps_cor)
        return 20 * np.log10(ps_cor[: L // 2])

    # FFT显示图像
    def get_FFTimage(self, x, y, i):
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        plt.subplot(2, 3, 1)
        plt.plot(x[0], y[0], color="blue", linewidth=2.5, linestyle="-", label="F_x")
        plt.title('第' + str(i) + '次切削的F_x的FFT图像')
        plt.subplot(2, 3, 2)
        plt.plot(x[1], y[1], color="blue", linewidth=2.5, linestyle="-", label="F_y")
        plt.title('第' + str(i) + '次切削的F_y的FFT图像')
        plt.subplot(2, 3, 3)
        plt.plot(x[2], y[2], color="blue", linewidth=2.5, linestyle="-", label="F_z")
        plt.title('第' + str(i) + '次切削的F_z的FFT图像')
        plt.subplot(2, 3, 4)
        plt.plot(x[3], y[3], color="blue", linewidth=2.5, linestyle="-", label="A_x")
        plt.title('第' + str(i) + '次切削的A_x的FFT图像')
        plt.subplot(2, 3, 5)
        plt.plot(x[4], y[4], color="blue", linewidth=2.5, linestyle="-", label="A_y")
        plt.title('第' + str(i) + '次切削的A_y的FFT图像')
        plt.subplot(2, 3, 6)
        plt.plot(x[5], y[5], color="blue", linewidth=2.5, linestyle="-", label="A_z")
        plt.title('第' + str(i) + '次切削的A_z的FFT图像')
        plt.savefig('D:\\image\\FFT.png')
        plt.show()

    # 自功率谱密度函数显示图像
    def get_SPSDFimage(self, y, i):
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        plt.subplot(2, 3, 1)
        plt.plot(y[0], color="blue", linewidth=2.5, linestyle="-", label="F_x")
        plt.title('第' + str(i) + '次切削的F_x的自功率谱密度函数')
        plt.subplot(2, 3, 2)
        plt.plot(y[1], color="blue", linewidth=2.5, linestyle="-", label="F_y")
        plt.title('第' + str(i) + '次切削的F_y的自功率谱密度函数')
        plt.subplot(2, 3, 3)
        plt.plot(y[2], color="blue", linewidth=2.5, linestyle="-", label="F_z")
        plt.title('第' + str(i) + '次切削的F_z的自功率谱密度函数')
        plt.subplot(2, 3, 4)
        plt.plot(y[3], color="blue", linewidth=2.5, linestyle="-", label="A_x")
        plt.title('第' + str(i) + '次切削的A_x的自功率谱密度函数')
        plt.subplot(2, 3, 5)
        plt.plot(y[4], color="blue", linewidth=2.5, linestyle="-", label="A_y")
        plt.title('第' + str(i) + '次切削的A_y的自功率谱密度函数')
        plt.subplot(2, 3, 6)
        plt.plot(y[5], color="blue", linewidth=2.5, linestyle="-", label="A_z")
        plt.title('第' + str(i) + '次切削的A_z的自功率谱密度函数')
        plt.savefig('D:\\image\\自功率谱密度函数.png')
        plt.show()


# 保存数据
def write_csv(sql_data, filename):
    file_name = filename
    save = pd.DataFrame(list(sql_data))
    try:
        save.to_csv(file_name)
    except UnicodeEncodeError:
        print("编码错误, 该数据无法写到文件中, 直接忽略该数据")

#————————————————————————————————————————————————————————————#
# 求时域指标
path = r'D:\PHM 2010\c1\c1'
GetFiles = GetDataFiles(path)
DataFiles = GetFiles.get_datafiles()

MEAN = [[], [], [], [], [], []]     # 均值
MS = [[], [], [], [], [], []]       # 均方值
RMS = [[], [], [], [], [], []]      # 均方根值
Var = [[], [], [], [], [], []]      # 方差
SD = [[], [], [], [], [], []]       # 标准差
MAX = [[], [], [], [], [], []]      # 最大值
MIN = [[], [], [], [], [], []]      # 最小值
DS = [[], [], [], [], [], []]       # 偏斜度
K = [[], [], [], [], [], []]        # 峭度
DSI = [[], [], [], [], [], []]      # 偏斜度指标
KI = [[], [], [], [], [], []]       # 峭度指标
WI = [[], [], [], [], [], []]       # 波形指标
PeI = [[], [], [], [], [], []]      # 峰值指标
PuI = [[], [], [], [], [], []]      # 脉冲指标
MI = [[], [], [], [], [], []]       # 裕度指标

k = 0
startime = time.perf_counter()

for f in DataFiles:
    # 导入数据
    data = pd.read_csv(f, header = None)
    # 进度条
    k = k + 1
    if k % 1 == 0:
        print('已经开始计算到第%d个'%k)
    # 时域分析
    TP = TimeDomainProcessing(data)
    for i in range(6):
        MEAN[i] = TP.get_Mean(i, MEAN[i])
        MS[i] = TP.get_MeanSquareValue(i, MS[i])
        RMS[i] = TP.get_RMS(i, RMS[i])
        Var[i] = TP.get_Variance(i, Var[i])
        SD[i] = TP.get_StandardDeviation(i, SD[i])
        MAX[i], MIN[i] = TP.get_Peak(i, MAX[i], MIN[i])
        DS[i] = TP.get_Degree_of_Skewness(i, DS[i])
        K[i] = TP.get_Kurtosis(i, K[i])
        DSI[i] = TP.get_DS_index(i, DSI[i])
        KI[i] = TP.get_Kurtosis_index(i, KI[i])
        WI[i] = TP.get_Waveform_index(i, WI[i])
        PeI[i] = TP.get_Peak_index(i, PeI[i])
        PuI[i] = TP.get_Pulse_index(i, PuI[i])
        MI[i] = TP.get_Margin_index(i, MI[i])
    endtime = time.perf_counter()
    print('用时：' + str(endtime - startime))
print('finish, no error')
# 写入文件
write_csv(MEAN, 'MEAN.csv')
write_csv(MS, 'MS.csv')
write_csv(RMS, 'RMS.csv')
write_csv(Var, 'Var.csv')
write_csv(SD, 'SD.csv')
write_csv(MAX, 'MAX.csv')
write_csv(MIN, 'MIN.csv')
write_csv(DS, 'DS.csv')
write_csv(K, 'K.csv')
write_csv(DSI, 'DSI.csv')
write_csv(KI, 'KI.csv')
write_csv(WI, 'WI.csv')
write_csv(PeI, 'PeI.csv')
write_csv(PuI, 'PuI.csv')
write_csv(MI, 'MI.csv')

#————————————————————————————————————————————————————————————#
# 时域指标显示(更改data_1的文件路径、TP.get_CVimage(y, '')处的内容)
path = r'D:\PHM 2010\c1\c1\c_1_001.csv'
data = pd.read_csv(path, header = None)
data_1 = pd.read_csv('D:\Workspaces\PyCharm\SignalProcessing_Project1\MEAN.csv')
data_1 = np.transpose(data_1)
data_1 = np.delete(data_1.values, 0, axis=0)
TP = TimeDomainProcessing(data)
y = []
for i in range(6):
    y.append(data_1[:, i])
TP.get_CVimage(y, '均值')

#————————————————————————————————————————————————————————————#
# 求互相关函数(更改data_1的文件路径、TP.get_CVimage(y, '')处的内容)
path = r'D:\PHM 2010\c1\c1\c_1_001.csv'
data = pd.read_csv(path, header = None)
data_1 = pd.read_csv('D:\Workspace\Pycharm\SignalProcessing_Project1\MEAN.csv')
data_2 = pd.read_csv('D:\PHM 2010\c1\c1_wear.csv')
data_1 = np.transpose(data_1)
data_1 = np.delete(data_1.values, 0, axis=0)
data2 = (data_2.values[:, 1] + data_2.values[:, 2] + data_2.values[:, 3])/3
TP = TimeDomainProcessing(data)
y = []
for i in range(6):
    data1 = data_1[:, i]
    y.append(TP.get_CrossCorrelation(data1, data2))
TP.get_CCimage(y, '均值')

#————————————————————————————————————————————————————————————#
# 求FFT及自功率谱函数（更改path的文件路径、FDP.get_FFTimage(,,i)\FDP.get_SPSDFimage(,i)）
Fs = 50000
path = r'D:\PHM 2010\c1\c1\c_1_001.csv'
data = pd.read_csv(path, header = None)
FDP = FrequencyDomainProcessing(data, Fs)
x1 = []
y1 = []
y2 = []
for i in range(6):
    x, y = FDP.get_FFT(i)
    yy = FDP.get_SelfPowerSpectrumDensityFunction(i)
    x1.append(x)
    y1.append(y)
    y2.append(yy)
FDP.get_FFTimage(x1, y1, 1)
FDP.get_SPSDFimage(y2, 1)